# Parallelized FindOrb, Prototyping Notebook

In [1]:
import pandas as pd

## Utils to read/write PSV ADES

In [2]:
def read_psv_ades(fn):
    """Read PSV ADES file
    
    Args:
        fn (str): Filename
        
    Returns:
        tuple: loaded dataframe, header
    """
    header = []
    with open(fn) as fp:
        # consume and store the header
        while True:
            line = fp.readline()
            if line[0] in ['#', '!']:
                header.append(line.rstrip())
                continue
            # the line is the header
            names = [ s.strip() for s in line.split('|') ]
            break
        df = pd.read_csv(fp, sep='|', header=0, names=names)
    return df, header

## df, hdr = read_psv_ades(fn)
## del df["rmsMag"] # workaround for FindOrb bug
## hdr
## df

In [3]:
def write_psv_ades(fn, df, header):
    """Write PSV ADES file
    
    Args:
        fn (str): Output filename
        df (pd.DataFrame): Dataframe of observations
        header (list): ADES file header (list of lines)
        
    """
    with open(fn, "w") as fp:
        fp.write('\n'.join(header))
        fp.write('\n')
        df.to_csv(fp, sep='|', index=False)

## t1 = df[df["trkSub"] == 't0000000']
## write_psv_ades("foo.psv", t1, hdr)

In [4]:
df, hdr = read_psv_ades("2022-10-18T09:06:40.464Z_discoveries_01_LSST_TEST.psv")
del df["rmsMag"] # workaround for FindOrb bug
df

,trkSub,obsTime,ra,dec,rmsRA,rmsDec,mag,band,stn,mode,astCat
0,t0000000,2022-10-04T04:31:35.818Z,340.215384,-31.237961,0.024390,0.028525,21.200,y,I11,CCD,Gaia2
1,t0000000,2022-10-08T04:57:44.150Z,339.602568,-31.208910,0.030950,0.036187,21.247,y,I11,CCD,Gaia2
2,t0000000,2022-10-08T05:20:55.363Z,339.600181,-31.208704,0.031220,0.036502,21.253,z,I11,CCD,Gaia2
3,t0000000,2022-10-08T05:42:49.939Z,339.597960,-31.208498,0.039912,0.046665,21.253,z,I11,CCD,Gaia2
4,t0000000,2022-10-08T06:06:00.720Z,339.595550,-31.208269,0.049422,0.057784,21.248,y,I11,CCD,Gaia2
...,...,...,...,...,...,...,...,...,...,...,...
455044,t0030307,2022-10-17T01:27:39.254Z,341.016670,-13.647847,0.107355,0.110474,22.814,i,I11,CCD,Gaia2
455045,t0030307,2022-10-18T00:39:11.635Z,340.963762,-13.645669,0.030286,0.031165,22.952,r,I11,CCD,Gaia2
455046,t0030307,2022-10-18T00:39:50.256Z,340.963714,-13.645656,0.029608,0.030468,22.952,r,I11,CCD,Gaia2
455047,t0030307,2022-10-18T01:02:39.437Z,340.962809,-13.645616,0.050079,0.051533,23.427,g,I11,CCD,Gaia2


## Multiprocess-safe FindOrb Wrappers

In [8]:
def fit_orbit(df, hdr):
    """Fit orbit with FindOrb to a single track in a multi-processing safe manner
    
    Args:
        df (pd.DataFrame): The observations to fit
        hdr (str): ADES file header for the observations in `df`
        
    Returns:
        dict: A dictionary with the result of the run, of the form::
        
            {
                'name': track name,
                'state_vec': state vector
                'epoch': state vector epoch
                'covar': covariance matrix
                'findorb': { # run details, useful for debugging
                    'args': how findorb was invoked
                    'returncode': UNIX return code (zero for success)
                    'stdout': the contents of Find_Orb standard output
                    'stderr': the contents of Find_Orb standard error
                }
            }
    """
    # make sure there's only a single tracklet
    assert len(df) >= 1
    assert len(df["trkSub"].unique()) == 1
    trkSub = df["trkSub"].iloc[0]

    # create a temporary directory
    import tempfile, subprocess, json, os
    with tempfile.TemporaryDirectory() as tmpdir:
        print(f"tmpdir: {tmpdir}")

        # prep the new "home" directory
        from shutil import copytree, ignore_patterns
        copytree(
            os.path.expanduser("~/.find_orb"), f"{tmpdir}/.find_orb",
            ignore=ignore_patterns('debug.txt', 'elements.json', 'elem_short.json', 'linux_p1550p2650.430t')
        )
        os.symlink(os.path.expanduser("~/.find_orb/linux_p1550p2650.430t"), f"{tmpdir}/.find_orb/linux_p1550p2650.430t")

        # dump to ades
        datafile = f"{tmpdir}/data.psv"
        resultdir = f"{tmpdir}/result"
        write_psv_ades(datafile, df, hdr)

        # call findorb
        cmd = f"fo {datafile} -O {resultdir} -D /Users/mjuric/projects/elasticsky/environ.dat"
        env = os.environ.copy()
        env["HOME"] = tmpdir
        ret = subprocess.run(cmd, shell=True, env=env, check=False, capture_output=True)

        # fetch/construct the result
        if ret.returncode == 0:
            # read the result
            with open(f"{resultdir}/covar.json") as fp:
                result = json.load(fp)
        else:
            result = {}

        result["name"] = trkSub
        result["findorb"] = {
            'args': ret.args,
            'returncode': ret.returncode,
            'stdout': ret.stdout.decode('utf-8'),
            'stderr': ret.stderr.decode('utf-8')
        }

        return result

## t1 = df[df["trkSub"] == 't0000000']
## orbit = fit_orbit(t1, hdr)
## orbit

In [41]:
%%time
def fit_orbit_batch(df, hdr, tracks=None):
    """Fit a batch of tracklets with FindOrb in a multi-processing safe manner

    These are all still processed within a single thread (process).

    Args:
        df (pd.DataFrame): The observations to fit
        hdr (str): ADES file header for the observations in `df`
        tracks (list): A list of trkSubs to fit; if None, process all from `df`
        
    Returns:
        dict: A dictionary of (trkSub: result), where result is the output of `fit_orbit`
    """
    if tracks is None:
        tracks = df['trkSub'].unique()

    results = []
    for trkSub in tracks:
        trk = df[df["trkSub"] == trkSub]
        results.append(fit_orbit(trk, hdr))
    return results

## orbits = fit_orbit_batch(df, hdr, df["trkSub"].unique()[:10])

CPU times: user 8 µs, sys: 3 µs, total: 11 µs
Wall time: 16.2 µs


## Parallelization with Ray

In [16]:
import ray
ray.init()

2020-12-09 17:21:20,050	INFO services.py:1090 -- View the Ray dashboard at http://127.0.0.1:8265


{'node_ip_address': '10.0.1.10',
 'raylet_ip_address': '10.0.1.10',
 'redis_address': '10.0.1.10:6379',
 'object_store_address': '/tmp/ray/session_2020-12-09_17-21-19_496480_27232/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2020-12-09_17-21-19_496480_27232/sockets/raylet',
 'webui_url': '127.0.0.1:8265',
 'session_dir': '/tmp/ray/session_2020-12-09_17-21-19_496480_27232',
 'metrics_export_port': 61492,
 'node_id': '62f2d5f7d1d1c844a1d787bbbe9a5cb609186dcf'}

Ray variant of the batched fitter:

In [46]:
@ray.remote
def dist_fit_orbit_batch(df, hdr, tracks):
    return fit_orbit_batch(df, hdr, tracks)

In [47]:
# A utility to divide up the tracklets into smaller chunks
def chunk(k, chunk_size):
     return[ k[i:i + chunk_size] for i in range(0, len(k), chunk_size) ]

#tracks = chunk(df['trkSub'].unique(), chunk_size=10)
#print(len(tracks))
#tracks[-1][-10:]

In [48]:
%%time

def processAdesFile(fn, chunk_size=10, max_tracklets=None):
    # load the file
    df, hdr = read_psv_ades(fn)
    del df["rmsMag"] # workaround for FindOrb bug

    # subdivide it into smaller chunks, with N tracklets each. These
    # chunks will be submitted to individual FindOrb threads to work on.
    tracks = chunk(df['trkSub'].unique(), chunk_size)

    # launch the parallel processing, and wait for the result
    df_id = ray.put(df)
    futures = [
        dist_fit_orbit_batch.remote(df_id, hdr, track_batch)
        for track_batch in tracks[:max_tracklets]
    ]
    chunked_results = ray.get(futures)
    del df_id

    # merge the result chunks
    results = [result for chunk in chunked_results for result in chunk]

    return results

# grab this file from https://epyc.astro.washington.edu/~moeyensj/rubin_submissions/ver5/
results = processAdesFile("2022-10-18T09:06:40.464Z_discoveries_01_LSST_TEST.psv", max_tracklets=10)

(pid=27395) tmpdir: /var/folders/0n/3cchn_wj5619fqz4dswygvj80000gn/T/tmpwsx2i23f
(pid=27398) tmpdir: /var/folders/0n/3cchn_wj5619fqz4dswygvj80000gn/T/tmps_1zrxnl
(pid=27399) tmpdir: /var/folders/0n/3cchn_wj5619fqz4dswygvj80000gn/T/tmpf_0xu2ri
(pid=27397) tmpdir: /var/folders/0n/3cchn_wj5619fqz4dswygvj80000gn/T/tmpvyf7_2_t
(pid=27396) tmpdir: /var/folders/0n/3cchn_wj5619fqz4dswygvj80000gn/T/tmp68fg1s75
(pid=27394) tmpdir: /var/folders/0n/3cchn_wj5619fqz4dswygvj80000gn/T/tmpc1zbmfxk
(pid=27392) tmpdir: /var/folders/0n/3cchn_wj5619fqz4dswygvj80000gn/T/tmp5gveu4qn
(pid=27393) tmpdir: /var/folders/0n/3cchn_wj5619fqz4dswygvj80000gn/T/tmp0yqw8vno
(pid=27392) tmpdir: /var/folders/0n/3cchn_wj5619fqz4dswygvj80000gn/T/tmp81pagu0l
(pid=27395) tmpdir: /var/folders/0n/3cchn_wj5619fqz4dswygvj80000gn/T/tmp9jq8fqj4
(pid=27394) tmpdir: /var/folders/0n/3cchn_wj5619fqz4dswygvj80000gn/T/tmpheubj0ts
(pid=27397) tmpdir: /var/folders/0n/3cchn_wj5619fqz4dswygvj80000gn/T/tmp5gn5mcl8
(pid=27398) tmpdir: /var/fol

(pid=27396) tmpdir: /var/folders/0n/3cchn_wj5619fqz4dswygvj80000gn/T/tmpta9n3mp1
(pid=27392) tmpdir: /var/folders/0n/3cchn_wj5619fqz4dswygvj80000gn/T/tmppfbg80_2
(pid=27396) tmpdir: /var/folders/0n/3cchn_wj5619fqz4dswygvj80000gn/T/tmpdahfslek
(pid=27396) tmpdir: /var/folders/0n/3cchn_wj5619fqz4dswygvj80000gn/T/tmpuz5o4guh
(pid=27392) tmpdir: /var/folders/0n/3cchn_wj5619fqz4dswygvj80000gn/T/tmpj63s7a1g
(pid=27392) tmpdir: /var/folders/0n/3cchn_wj5619fqz4dswygvj80000gn/T/tmpxo3xuy1k
(pid=27396) tmpdir: /var/folders/0n/3cchn_wj5619fqz4dswygvj80000gn/T/tmpnbzwlu9e
(pid=27392) tmpdir: /var/folders/0n/3cchn_wj5619fqz4dswygvj80000gn/T/tmpj8_y6x5l
(pid=27396) tmpdir: /var/folders/0n/3cchn_wj5619fqz4dswygvj80000gn/T/tmp30zkvedp
(pid=27392) tmpdir: /var/folders/0n/3cchn_wj5619fqz4dswygvj80000gn/T/tmp6eu9gttc
(pid=27396) tmpdir: /var/folders/0n/3cchn_wj5619fqz4dswygvj80000gn/T/tmptbreo1qz
(pid=27392) tmpdir: /var/folders/0n/3cchn_wj5619fqz4dswygvj80000gn/T/tmphrmlc533
(pid=27396) tmpdir: /var/fol

In [50]:
len(results)

100

## Streaming implementation

In [96]:
class BatchRun:
    def __init__(self, fn):
        df, hdr = read_psv_ades(fn)
        del df["rmsMag"] # workaround for FindOrb bug

        self.df, self.hdr = df, hdr

    def start(self, chunk_size=10, max_tracklets=None):
        # subdivide the data into smaller chunks, with N tracklets each. These
        # will be submitted to individual FindOrb threads to work on.
        tracks = chunk(df['trkSub'].unique(), chunk_size)

        # launch the parallel processing
        df_id = ray.put(df)
        self.tasks = [
            dist_fit_orbit_batch.remote(df_id, hdr, track_batch)
            for track_batch in tracks[:max_tracklets]
        ]
        self.result = []

    def collect(self, num_returns=None, timeout=0):
        if num_returns==None:
            num_returns = len(self.tasks)

        # collect the results that have finished
        done, tasks = ray.wait(self.tasks, num_returns=num_returns, timeout=timeout)
        
        chunked_results = ray.get(done)
        results = [result for chunk in chunked_results for result in chunk]
        
        self.result += results
        self.tasks = tasks

        return len(results)

# grab this file from https://epyc.astro.washington.edu/~moeyensj/rubin_submissions/ver5/
batch = BatchRun("2022-10-18T09:06:40.464Z_discoveries_01_LSST_TEST.psv")
batch.start(10, 10)

(pid=27395) tmpdir: /var/folders/0n/3cchn_wj5619fqz4dswygvj80000gn/T/tmprmuv21dp
(pid=27398) tmpdir: /var/folders/0n/3cchn_wj5619fqz4dswygvj80000gn/T/tmpv77983qi
(pid=27399) tmpdir: /var/folders/0n/3cchn_wj5619fqz4dswygvj80000gn/T/tmp7so7qf39
(pid=27397) tmpdir: /var/folders/0n/3cchn_wj5619fqz4dswygvj80000gn/T/tmpwd6x90bp
(pid=27396) tmpdir: /var/folders/0n/3cchn_wj5619fqz4dswygvj80000gn/T/tmpa5mam1uo
(pid=27394) tmpdir: /var/folders/0n/3cchn_wj5619fqz4dswygvj80000gn/T/tmp74jg9ypy
(pid=27392) tmpdir: /var/folders/0n/3cchn_wj5619fqz4dswygvj80000gn/T/tmptgndujhy
(pid=27393) tmpdir: /var/folders/0n/3cchn_wj5619fqz4dswygvj80000gn/T/tmpqs76m6cu


In [97]:
len(batch.tasks)

(pid=27399) tmpdir: /var/folders/0n/3cchn_wj5619fqz4dswygvj80000gn/T/tmpljfyvip7


10

(pid=27395) tmpdir: /var/folders/0n/3cchn_wj5619fqz4dswygvj80000gn/T/tmphyndtnl7
(pid=27393) tmpdir: /var/folders/0n/3cchn_wj5619fqz4dswygvj80000gn/T/tmphjni275w
(pid=27392) tmpdir: /var/folders/0n/3cchn_wj5619fqz4dswygvj80000gn/T/tmp6_wj06gg
(pid=27394) tmpdir: /var/folders/0n/3cchn_wj5619fqz4dswygvj80000gn/T/tmpplemugzo
(pid=27398) tmpdir: /var/folders/0n/3cchn_wj5619fqz4dswygvj80000gn/T/tmpoyudm786
(pid=27396) tmpdir: /var/folders/0n/3cchn_wj5619fqz4dswygvj80000gn/T/tmpbcuvg786
(pid=27397) tmpdir: /var/folders/0n/3cchn_wj5619fqz4dswygvj80000gn/T/tmpk0v6w5o2


In [105]:
batch.collect()

2

In [69]:
res = 1
while res:
    res = batch.collect()

In [106]:
len(batch.result)

100

In [112]:
orbits = batch.result

## Q/A

Check for failiures:

In [113]:
failures = [ result for result in orbits if result['findorb']['returncode'] != 0]
print(len(failures))
failures

0


[]

In [136]:
def to_fwf(fn, df):
    from tabulate import tabulate
    content = tabulate(df.values.tolist(), list(df.columns), tablefmt="plain", floatfmt=".12f")
    with open(fn, "w") as fp:
        fp.write(content)
        fp.write("\n")

Construct a dataframe of states:

59856.18942099996

In [141]:
states = [ result['state_vect'] for result in orbits ]
epochs = [ result["epoch"] - 2400000.5 for result in orbits]
keys = [ result['name'] for result in orbits ]
states = pd.DataFrame(states, index=keys, columns=["x", "y", "z", "vx", "vy", "vz"])
states["epoch"] = epochs
states.reset_index(inplace=True)
states = states.rename(columns = {'index':'trkSub'})
states.sort_index()
to_fwf("result.txt", states)
states

,trkSub,x,y,z,vx,vy,vz,epoch
0,t0000000,3.732200,-1.427343,-1.232355,0.001064,0.005572,-0.000092,59856.189421
1,t0000001,4.013681,0.824397,-1.523476,-0.000825,0.002680,-0.000553,59859.291354
2,t0000002,2.375318,-0.443731,0.379375,0.001031,0.008871,0.001200,59855.072901
3,t0000003,1.253650,-0.359243,-0.265275,0.009259,0.013286,-0.002807,59858.082350
4,t0000004,1.620572,-0.258156,0.222494,0.014663,0.008725,-0.000178,59855.042805
...,...,...,...,...,...,...,...,...
95,t0000095,1.513769,-0.277918,0.038995,0.009665,0.012959,-0.000895,59855.016054
96,t0000096,1.213115,-0.028367,-0.137101,-0.015274,0.011064,-0.000784,59859.014573
97,t0000097,1.698212,-0.096381,-1.087345,0.008248,0.007391,-0.007025,59857.285042
98,t0000098,1.569186,-0.360982,-0.250207,0.010196,0.010001,-0.002535,59860.200852


In [85]:
# states1 = states

In [86]:
(states1 == states).all()

0    True
1    True
2    True
3    True
4    True
5    True
dtype: bool